In [1]:
import pandas as pd
import numpy as np

medal_counts = pd.read_csv('2025_Problem_C_Data/summerOly_medal_counts_utf8.csv')

In [15]:
# Medal counts for year 2024
medal_counts_2024 = medal_counts[medal_counts['Year'] > 2000]
medal_counts_2024

,Rank,NOC,Gold,Silver,Bronze,Total,Year
918,1,United States,36,39,26,101,2004
919,2,China,32,17,14,63,2004
920,3,Russia,28,26,36,90,2004
921,4,Australia,17,16,17,50,2004
922,5,Japan,16,9,12,37,2004
...,...,...,...,...,...,...,...
1430,84,Qatar,0,0,1,1,2024
1431,84,Refugee Olympic Team,0,0,1,1,2024
1432,84,Singapore,0,0,1,1,2024
1433,84,Slovakia,0,0,1,1,2024


In [19]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az

# Sample data (replace with your full dataset)
data = {
    "USA": {
        2016: [46, 37, 38, 121, 500, 30],  # [gold, silver, bronze, total, athletes, events]
        2020: [39, 41, 33, 113, 550, 32],
        2024: [40, 44, 42, 126, 600, 35]
    },
    "China": {
        2016: [26, 18, 26, 70, 400, 25],
        2020: [38, 32, 18, 88, 450, 28],
        2024: [40, 27, 24, 91, 480, 30]
    }
}

# Convert to DataFrame
rows = []
for country, years in data.items():
    for year, values in years.items():
        rows.append({
            "country": country,
            "year": year,
            "gold": values[0],
            "total": values[3],
            "athletes": values[4],
            "events": values[5],
            "host": 1 if (year == 2024 and country == "France") or (year == 2028 and country == "USA") else 0
        })
df = pd.DataFrame(rows)

# Add lagged medals from the previous Olympics
df = df.sort_values(["country", "year"])
df["lag_gold"] = df.groupby("country")["gold"].shift(1)
df["lag_total"] = df.groupby("country")["total"].shift(1)
df = df.dropna()  # Remove rows with missing lagged data

/Users/yubraj/.pyenv/versions/3.8.10/lib/python3.8/site-packages/_distutils_hack/__init__.py:16: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/yubraj/.pyenv/versions/3.8.10/lib/python3.8/site-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [20]:
countries = df["country"].unique()
n_countries = len(countries)
country_idx = pd.Categorical(df["country"]).codes  # Integer indices for countries

with pm.Model() as model_gold:
    # Hyperpriors for group-level effects
    mu_a = pm.Normal("mu_a", mu=0, sigma=10)
    sigma_a = pm.HalfNormal("sigma_a", sigma=10)
    
    # Non-centered parameterization for country intercepts
    a_country_raw = pm.Normal("a_country_raw", mu=0, sigma=1, shape=n_countries)
    a_country = pm.Deterministic("a_country", mu_a + a_country_raw * sigma_a)
    
    # Coefficients for predictors
    b_lag = pm.Normal("b_lag", mu=0, sigma=5)
    b_host = pm.Normal("b_host", mu=0, sigma=5)
    b_athletes = pm.Normal("b_athletes", mu=0, sigma=5)
    b_events = pm.Normal("b_events", mu=0, sigma=5)
    
    # Linear predictor
    mu = (
        a_country[country_idx] +
        b_lag * df["lag_gold"] +
        b_host * df["host"] +
        b_athletes * df["athletes"] +
        b_events * df["events"]
    )
    
    # Negative Binomial likelihood (for overdispersion)
    alpha = pm.HalfNormal("alpha", sigma=5)
    gold_medals = pm.NegativeBinomial(
        "gold_medals", 
        mu=pm.math.exp(mu),  # Ensure positivity
        alpha=alpha, 
        observed=df["gold"]
    )

In [21]:
with model_gold:
    trace_gold = pm.sample(500, tune=1000, target_accept=0.9, return_inferencedata=True)

# Check convergence
az.summary(trace_gold, var_names=["mu_a", "sigma_a", "b_lag", "b_host", "b_athletes", "b_events"])

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu_a, sigma_a, a_country_raw, b_lag, b_host, b_athletes, b_events, alpha]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 247 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
There were 70 divergences after tuning. Increase `target_accept` or reparameterize.
Chain 0 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain 1 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain 2 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain 3 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu_a,1.715,6.932,-12.345,14.033,0.232,0.164,924.0,793.0,1.01
sigma_a,5.824,4.891,0.002,14.889,0.187,0.132,526.0,653.0,1.02
b_lag,-0.000,0.089,-0.143,0.154,0.003,0.003,1078.0,1290.0,1.01
b_host,0.244,5.156,-10.242,8.621,0.311,0.220,202.0,527.0,1.03
b_athletes,-0.002,0.159,-0.327,0.303,0.008,0.006,398.0,515.0,1.01
b_events,0.076,2.627,-4.892,5.411,0.127,0.090,413.0,541.0,1.01


In [22]:
# Prepare 2028 data (example for USA)
usa_2028 = pd.DataFrame({
    "country": ["USA"],
    "lag_gold": [40],  # 2024 gold medals
    "host": [1],       # USA is the 2028 host
    "athletes": [620], # Hypothetical athlete count
    "events": [36]     # Hypothetical events participated in
})

country_idx_2028 = pd.Categorical(usa_2028["country"], categories=countries).codes[0]

with model_gold:
    # Create posterior predictive samples
    pm.set_data({
        "a_country_raw": trace_gold.posterior["a_country_raw"].values[:, :, country_idx_2028],
        "lag_gold": usa_2028["lag_gold"],
        "host": usa_2028["host"],
        "athletes": usa_2028["athletes"],
        "events": usa_2028["events"]
    })
    pred_gold = pm.sample_posterior_predictive(trace_gold, var_names=["gold_medals"])

TypeError: The variable `a_country_raw` must be a `SharedVariable` (created through `pm.MutableData()` or `pm.Data(mutable=True)`) to allow updating. The current type is: <class 'pytensor.tensor.var.TensorVariable'>

In [ ]:
import matplotlib.pyplot as plt

# Plot posterior distribution of USA's 2028 gold medals
az.plot_posterior(pred_gold["gold_medals"], hdi_prob=0.9)
plt.title("USA 2028 Gold Medal Predictions")
plt.xlabel("Gold Medals")
plt.show()